EFFICIENTNET ANTISPOOF MODEL TRAINED BY ANANDHU
https://github.com/Anandhu1228

In [9]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import timm
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score
import time
import numpy as np

In [10]:
dataset_path = "dataset_heavy_augment" 
train_dir = os.path.join(dataset_path, "train")
val_dir = os.path.join(dataset_path, "val")

from torchvision.datasets import ImageFolder

train_dataset = ImageFolder(root=train_dir)
val_dataset = ImageFolder(root=val_dir)

print(f"Train images: {len(train_dataset)}")
print(f"Validation images: {len(val_dataset)}")

print("\nClass distribution in train:")
for class_idx, count in zip(*np.unique(train_dataset.targets, return_counts=True)):
    print(f"{train_dataset.classes[class_idx]}: {count}")

print("\nClass distribution in validation:")
for class_idx, count in zip(*np.unique(val_dataset.targets, return_counts=True)):
    print(f"{val_dataset.classes[class_idx]}: {count}")

Train images: 103851
Validation images: 18285

Class distribution in train:
real: 53053
spoof: 50798

Class distribution in validation:
real: 10750
spoof: 7535


In [11]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [12]:
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model("efficientnet_b0", pretrained=False, num_classes=2)

model = model.to(device)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00005)

In [15]:
checkpoint_dir = "runs/classify"
os.makedirs(checkpoint_dir, exist_ok=True)

last_checkpoint_path = os.path.join(checkpoint_dir, "L1_last_checkpoint.pth")
best_acc_path = os.path.join(checkpoint_dir, "L1_best_acc_model.pth")
best_loss_path = os.path.join(checkpoint_dir, "L1_best_loss_model.pth")

best_val_acc = 0.0
best_val_loss = float('inf')

if os.path.exists(last_checkpoint_path):
    checkpoint = torch.load(last_checkpoint_path)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    start_epoch = checkpoint["epoch"] + 1
    best_val_acc = checkpoint.get("best_val_acc", 0.0)
    best_val_loss = checkpoint.get("best_val_loss", float('inf'))
    print(f"🔄 Resuming training from epoch {start_epoch}...")
else:
    start_epoch = 1

In [16]:
num_epochs = 180
for epoch in range(start_epoch, num_epochs + 1):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds, all_labels = [], []

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}", leave=False)
    
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        progress_bar.set_postfix(loss=running_loss/len(train_loader), acc=100.*correct/total)

    train_loss = running_loss / len(train_loader)
    train_acc = 100. * correct / total
    train_precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    train_recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    val_all_preds, val_all_labels = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_correct += predicted.eq(labels).sum().item()
            val_total += labels.size(0)
            val_all_preds.extend(predicted.cpu().numpy())
            val_all_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_acc = 100. * val_correct / val_total
    val_precision = precision_score(val_all_labels, val_all_preds, average='macro', zero_division=0)
    val_recall = recall_score(val_all_labels, val_all_preds, average='macro', zero_division=0)

    print(f"\n✔️ Epoch {epoch}:")
    print(f"Train -> Loss: {train_loss:.4f}    Precision: {train_precision:.4f}    Recall: {train_recall:.4f}    Acc: {train_acc:.2f}%")
    print(f"Valid -> Loss: {val_loss:.4f}    Precision: {val_precision:.4f}    Recall: {val_recall:.4f}    Acc: {val_acc:.2f}%")

    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "best_val_acc": best_val_acc,
        "val_acc": val_acc,
    }, last_checkpoint_path)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_acc_path)
        print(f"✅ New best model saved at epoch {epoch} (Val Acc: {val_acc:.2f}%)")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_loss_path)
        print(f"☑️ New low loss model saved at epoch {epoch} (Loss: {val_loss:.2f})")



✔️ Epoch 1:
Train -> Loss: 0.1729    Precision: 0.9347    Recall: 0.9349    Acc: 93.47%
Valid -> Loss: 0.8435    Precision: 0.8512    Recall: 0.8426    Acc: 85.26%
✅ New best model saved at epoch 1 (Val Acc: 85.26%)
☑️ New low loss model saved at epoch 1 (Loss: 0.84)



✔️ Epoch 2:
Train -> Loss: 0.0321    Precision: 0.9900    Recall: 0.9900    Acc: 99.00%
Valid -> Loss: 3.8368    Precision: 0.9055    Recall: 0.8652    Acc: 88.53%
✅ New best model saved at epoch 2 (Val Acc: 88.53%)



✔️ Epoch 3:
Train -> Loss: 0.0123    Precision: 0.9962    Recall: 0.9962    Acc: 99.62%
Valid -> Loss: 1.3357    Precision: 0.9241    Recall: 0.8817    Acc: 90.13%
✅ New best model saved at epoch 3 (Val Acc: 90.13%)



✔️ Epoch 4:
Train -> Loss: 0.0087    Precision: 0.9974    Recall: 0.9974    Acc: 99.74%
Valid -> Loss: 1.3909    Precision: 0.9265    Recall: 0.9033    Acc: 91.63%
✅ New best model saved at epoch 4 (Val Acc: 91.63%)



✔️ Epoch 5:
Train -> Loss: 0.0063    Precision: 0.9982    Recall: 0.9982    Acc: 99.82%
Valid -> Loss: 1.2647    Precision: 0.9342    Recall: 0.8933    Acc: 91.19%


...


✔️ Epoch 35:
Train -> Loss: 0.0004    Precision: 0.9998    Recall: 0.9998    Acc: 99.98%
Valid -> Loss: 3.3890    Precision: 0.9104    Recall: 0.8443    Acc: 87.16%



✔️ Epoch 36:
Train -> Loss: 0.0000    Precision: 1.0000    Recall: 1.0000    Acc: 100.00%
Valid -> Loss: 3.3578    Precision: 0.9071    Recall: 0.8373    Acc: 86.59%



✔️ Epoch 37:
Train -> Loss: 0.0000    Precision: 1.0000    Recall: 1.0000    Acc: 100.00%
Valid -> Loss: 3.6764    Precision: 0.9096    Recall: 0.8425    Acc: 87.02%



✔️ Epoch 38:
Train -> Loss: 0.0000    Precision: 1.0000    Recall: 1.0000    Acc: 100.00%
Valid -> Loss: 3.5679    Precision: 0.9079    Recall: 0.8390    Acc: 86.73%


KeyboardInterrupt: 

In [ ]:
timestamp = time.strftime("%Y%m%d_%H%M%S")
final_model_path = os.path.join(checkpoint_dir, f"final_model_{timestamp}.pth")
torch.save(model.state_dict(), final_model_path)

if os.path.exists(last_checkpoint_path):
    os.remove(last_checkpoint_path)

print(f"\n🎯 Training Complete!")
print(f"Best Accuracy Model: {best_val_acc:.2f}% (saved to {best_acc_path})")
print(f"Best Loss Model: {best_val_loss:.4f} (saved to {best_loss_path})")
print(f"Final Model: {final_model_path}")